In [104]:
import pandas as pd 
import pandas_profiling as pp 
aguadataframe = pd.read_csv('Calidad_de_agua_2022.csv',delimiter = ';')
aguadataframe
aguadataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168 entries, 0 to 167
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   orden                    168 non-null    int64 
 1   sitios                   168 non-null    object
 2   codigo                   168 non-null    object
 3   fecha                    168 non-null    object
 4   año                      168 non-null    object
 5   campaña                  168 non-null    object
 6   tem_agua                 168 non-null    object
 7   tem_aire                 166 non-null    object
 8   od                       168 non-null    object
 9   ph                       168 non-null    object
 10  olores                   168 non-null    object
 11  color                    168 non-null    object
 12  espumas                  168 non-null    object
 13  mat_susp                 168 non-null    object
 14  colif_fecales_ufc_100ml  167 non-null    o

viendo la informacion del dataset, se puede ver que hay algunas columnas que contienen valores nulos (informacion para un posterior analisis) y ademas que todas las columnas son de tipo objet lo cual nos da un indicio de que algunas columnas vamos a tener que transformarlas y cambiarlas de tipo. 

In [105]:
aguadataframe = aguadataframe.drop(columns='orden')
aguadataframe = aguadataframe.drop(columns='año')
aguadataframe = aguadataframe.drop(columns='campaña')
aguadataframe = aguadataframe.drop(columns='cd_total_mg_l')
aguadataframe = aguadataframe.drop(columns='calidad_de_agua')
aguadataframe = aguadataframe.drop(columns='hidr_deriv_petr_ug_l')
aguadataframe = aguadataframe.drop(columns='codigo')

tomamos la decision de eliminar la columna orden debido a que no aportaba ningun tipo de informacion para el analisis de los datos. 

In [106]:
aguadataframe.columns
aguadataframe.describe()

,sitios,fecha,tem_agua,tem_aire,od,ph,olores,color,espumas,mat_susp,...,nh4_mg_l,p_total_l_mg_l,fosf_ortofos_mg_l,dbo_mg_l,dqo_mg_l,turbiedad_ntu,cr_total_mg_l,clorofila_a_ug_l,microcistina_ug_l,ica
count,168,168,168,166,168,168,168,168,168,168,...,168,168,168,167,168,168,168,167,168,155
unique,42,7,93,32,130,116,6,6,6,7,...,87,75,68,55,51,63,27,70,15,39
top,Canal Villanueva y Río Luján,23/2/2022,no se midió,14,no se midió,no se midió,Ausencia,Ausencia,Ausencia,Ausencia,...,<0.05,no se midió,no se midió,no se midió,<30,no se midió,<0.005,<10,<0.15,40
freq,4,42,19,25,24,23,120,119,127,106,...,16,17,10,53,51,10,113,74,76,13


como vemos cuando hacemos un describe nos muestra pocos datos y omite los estadisticos como la media, los cuartiles, etc. Esto es debido a que todas las variables son de tipo object asique lo que habria que hacer es convertirlas a algun otro tipo para poder analizarlas. 

In [107]:
aguadataframe1 = aguadataframe.copy()
columnas_numericas=['tem_agua', 'tem_aire', 'od', 'ph','turbiedad_ntu','ica']
aguadataframe1[columnas_numericas]=aguadataframe[columnas_numericas].replace(',', '.', regex=True).apply(pd.to_numeric, errors='coerce')


In [108]:
columnas_ausencia_presencia = ['olores','espumas','color','mat_susp']
aguadataframe1 = aguadataframe1.replace({"no se midió": pd.NA,
                                         "no midieron este día": pd.NA,
                                         "no se determinó": pd.NA,
                                         "no midio la sonda": pd.NA})
aguadataframe1 = aguadataframe1.dropna(how='all', subset=columnas_ausencia_presencia)

In [109]:
boolean_columns = ['espumas', 'olores', 'color', 'mat_susp']
for col in boolean_columns:
    aguadataframe1[col] = aguadataframe1[col].replace({'Presencia': True,
                                                       'Ausencia': False,
                                                       'ausencia': False,
                                                       'Ausente': False,
                                                       }).astype(bool)



C:\Users\court\AppData\Local\Temp\ipykernel_12744\2023971720.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  aguadataframe1[col] = aguadataframe1[col].replace({'Presencia': True,
C:\Users\court\AppData\Local\Temp\ipykernel_12744\2023971720.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  aguadataframe1[col] = aguadataframe1[col].replace({'Presencia': True,
C:\Users\court\AppData\Local\Temp\ipykernel_12744\2023971720.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version.

ahora que transformamos los valores se puede ver un mejor analisis del metodo describe, por ejemplo se puede ver la media de la temperatura del agua que es de 17.82, el desvio, los valores maximos y minimos y los cuartiles.  Para las columnas que tienen valores que dicen "ausencia", "presencia", las vamos a convertir en booleanas para por ejemplo analizar la frecuencia o analizar algun tipo de correlacion con alguna otra variable.

una vez que ya pasamos todas las variables a un tipo mas analizable, podemos hacer la matriz de correlación para ver si hay variables que se relacionar por tener una correlación alta, eso si no siempre que tengan una correlacion alta significa que las variables están relacionadas

In [110]:
numeric_columns= aguadataframe1.select_dtypes(include=['float64','int64'])
correlation_matrix=numeric_columns.corr()
correlation_matrix

,tem_agua,tem_aire,od,ph,turbiedad_ntu,ica
tem_agua,1.000000,0.749406,-0.262059,-0.210221,-0.020988,0.270831
tem_aire,0.749406,1.000000,-0.174873,-0.164793,-0.071928,0.045179
od,-0.262059,-0.174873,1.000000,0.649507,0.218779,0.250481
ph,-0.210221,-0.164793,0.649507,1.000000,0.101756,0.027619
turbiedad_ntu,-0.020988,-0.071928,0.218779,0.101756,1.000000,0.174438
ica,0.270831,0.045179,0.250481,0.027619,0.174438,1.000000


In [111]:
aguadataframe1['turbiedad_ntu'].value_counts()

turbiedad_ntu
12.0     7
19.0     6
45.0     6
22.0     6
28.0     5
23.0     5
90.0     5
60.0     5
13.0     5
26.0     5
17.0     5
25.0     5
18.0     4
39.0     4
30.0     4
34.0     4
16.0     4
70.0     3
14.0     3
31.0     3
33.0     3
75.0     3
29.0     3
37.0     3
50.0     3
35.0     3
65.0     3
7.5      2
55.0     2
27.0     2
85.0     2
80.0     2
11.0     2
8.9      1
4.1      1
36.0     1
110.0    1
130.0    1
71.0     1
38.0     1
3.3      1
84.0     1
24.0     1
2.5      1
6.1      1
6.0      1
32.0     1
67.0     1
48.0     1
10.0     1
8.6      1
54.0     1
7.3      1
8.5      1
9.3      1
15.0     1
21.0     1
20.0     1
18.2     1
40.0     1
Name: count, dtype: int64

In [112]:
aguadataframe1.to_csv('aguitaFRESCA.csv')